<a href="https://colab.research.google.com/gist/FS-CS/65cf046eab8e975461565966216a0112/mnist_cnn_cgpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this demonstration notebook, we show how to obtain near-state-of-the-art performance on the well-known MNIST digit indentification dataset using light Convolutional Neural Network (CNN) architectures and simple methods with short training times. We use the deep learning library PyTorch.

We present the results for each new step taken in that direction in order to get a feeling of what does each improvement bring to the table. We visualize this progress with a dynamical monitoring and plotting of the relevant metrics during the training of the models.

We reach accuracies up to 99.70%, while state-of-the-art is 99.84% (results may slightly vary with different runs).

# Initialisation and imports
---

In [ ]:
#!pip3 install torch torchvision Pillow matplotlib

In [ ]:
import numpy as np
import copy
import random
import time
from itertools import cycle
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter, PercentFormatter, LogLocator
from IPython.display import display, clear_output
import ipywidgets as widgets

import torch
import torch.nn.functional as F
from torch.utils.data import sampler, DataLoader
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
import torch.nn as nn
from torch.optim.lr_scheduler import CosineAnnealingLR, CosineAnnealingWarmRestarts, CyclicLR

# Check if GPU is available
use_gpu = torch.cuda.is_available()
device = torch.device("cuda:0" if use_gpu else "cpu")

# Fixing random seed
manualSeed = 1234
random.seed(manualSeed)
np.random.seed(manualSeed)
torch.manual_seed(manualSeed)
if use_gpu:
   torch.cuda.manual_seed_all(manualSeed)

print("Torch version: ", torch.__version__)
print("GPU Available: {}".format(use_gpu))

Torch version:  1.6.0+cu101
GPU Available: True


# Data sampling
---

We use the `ChunkSampler` helper class to create our train, valid and test sets by selecting portions of the dataset

In [ ]:
class ChunkSampler(sampler.Sampler):
    """Samples elements sequentially from some offset.
    From: https://github.com/pytorch/vision/issues/168
    
    Parameters
    ----------
    num_samples: int
      # of desired datapoints
    start: int
      Offset where we should start selecting from
    """
    def __init__(self, num_samples, start=0):
        self.num_samples = num_samples
        self.start = start

    def __iter__(self):
        return iter(range(self.start, self.start + self.num_samples))

    def __len__(self):
        return self.num_samples

The `generate_data` helper function creates the train, valid and test datasets and dataloaders with the required transforms and batch size.

In [ ]:
def generate_data(batch_size=100, transform=transforms.ToTensor()):

    # We use the MNIST class from torchvision.datasets to download the MNIST data in train (train=True, download=True) and test sets
    train_dataset = MNIST(root='../data', 
                          train=True, 
                          transform=transform,  
                          download=True)

    test_dataset = MNIST(root='../data', 
                         train=False, 
                         transform=transforms.ToTensor())

    train_dataset_sizes = len(train_dataset)
    # We will use 80% of the train data to effectively train the model, the remaining 20% will be used for validation
    num_train_samples = int(0.8 * train_dataset_sizes)
    num_valid_samples = train_dataset_sizes - num_train_samples
    num_test_samples = len(test_dataset)

    #print('Number of train examples: {}'.format(num_train_samples))
    #print('Number of valid examples: {}'.format(num_valid_samples))
    #print('Number of test examples: {}'.format(num_test_samples))

    # We load the different datasets in DataLoader objects with the specified batch size
    train_loader = DataLoader(dataset=train_dataset,
                              sampler=ChunkSampler(num_train_samples, 0),
                              batch_size=batch_size, 
                              shuffle=False)

    valid_loader = DataLoader(dataset=train_dataset,
                              sampler=ChunkSampler(
                                  num_valid_samples, num_train_samples),
                              batch_size=batch_size, 
                              shuffle=False)

    test_loader = DataLoader(dataset=test_dataset, 
                             batch_size=batch_size, 
                             shuffle=False)
    
    return train_loader, valid_loader, test_loader

# MNIST train-test functions
---

The `train_test_MNIST` function trains and computes the train and valid losses as well as the accuracy on the test set. The arguments to provide are the network architecture to train, the initial learning rate and the required optimizer. In addition, a learning rate scheduler can be specified.

The function provides dynamical prints and plots to monitor the indicators (accuracy monitoring is optional and activated by setting `monitor_acc` to True). In order to visualize indicators during training, the `verbose` argument needs to be set to True.

The valid loss has its own axis since models with Dropout have signficantly larger train loss than valid loss, as Dropout is not applied on validation data. Therefore having the two on the same axis would crush the valid loss curve for these models and make it hard to visualize.

When using learning rate schedulers, it is possible to monitor the global learning rate (optimizers like Adam have a different learning rate for each parameter of the model) in a separate subplot by setting `monitor_lr` to True.

In [ ]:
def train_test_MNIST(network, num_epochs, learning_rate, optimizer=torch.optim.Adam,
                     monitor_acc=False, scheduler=None, monitor_lr=False,
                     verbose=False, **kwargs):

    # Initialize a new instance of the model architecture
    model = network()
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optimizer(model.parameters(), lr=learning_rate)
    if scheduler is not None:
        scheduler=scheduler(optimizer, **kwargs)

    # Create an output widget to dynamically display the loss values per epoch
    if verbose is True:
        print('\nStart training.\n')
        out = widgets.Output()
        display(out)
        plt.style.use('dark_background')
        fig = plt.figure(figsize=(15, 3))
        ax_main = fig.add_subplot(121)
        ax_main.set_yscale('log')
#        ax_main.yaxis.set_major_locator(LogLocator(base=10))
#        ax_main.yaxis.set_major_formatter(StrMethodFormatter('{x:.1f}'))
        ax_main.set_xlabel('Number of epochs')
        ax_main.set_ylabel('Train Loss')
        par1 = ax_main.twinx()
        par1.set_yscale('log')
        par1.set_ylabel('Valid loss')
        if monitor_acc is True:
            par2 = ax_main.twinx()
            par2.set_yscale('log')
            par2.set_ylabel('Accuracy')
            par2.spines["right"].set_position(("axes", 1.25))
            par2.spines["right"].set_visible(True)
#            par2.yaxis.set_major_formatter(PercentFormatter())
        if monitor_lr is True:
            ax_lr = fig.add_subplot(122)
            ax_lr.set_xlabel('Iterations over examples')
            ax_lr.set_ylabel('Learning rate')
            plt.subplots_adjust(wspace=1.3)

    train_loss_history, valid_loss_history, acc_history = [], [], []
    lr_history = []

    # Start time
    start_time = time.time()

    for epoch in range(num_epochs):

        train_loss = 0
        train_n_iter = 0

        if monitor_lr is True:
            lr_history = []

        # Set the model to train mode (weight optimization)
        model.train()

        # Iterate over the train data of the corresponding batch
        for images, labels in train_loader:

            # Load the data on the proper device
            images = images.to(device)
            labels = labels.to(device)

            # Set the gradients to zero
            optimizer.zero_grad()

            # Apply the model to the input images (forward)
            outputs = model(images)

            # Compute the loss with respect to the labels
            loss = criterion(outputs, labels)

            # Backward propagation
            loss.backward()

            # Optimize
            optimizer.step()

            # Scheduler for the learning rate
            if scheduler is not None:
                scheduler.step()
                if monitor_lr is True:
                    lr_history.append(scheduler.get_last_lr())
        
            # Statistics
            train_loss += loss.item()
            train_n_iter += 1
            
        train_loss_history.append(train_loss/train_n_iter)

        valid_loss = 0
        valid_n_iter = 0

        # Set the model to evaluate mode (no weight optimization)
        model.eval()

        with torch.no_grad():

            # Compute the loss on the validation set
            for images, labels in valid_loader:

                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)

                loss = criterion(outputs, labels)

                valid_loss += loss.item()
                valid_n_iter += 1

            valid_loss_history.append(valid_loss/valid_n_iter)

            if monitor_acc is True:

                correct, total = 0, 0

                for images, labels in test_loader:

                    images = images.to(device)
                    labels = labels.to(device)

                    outputs = model(images)

                    # Keep the top prediction
                    _, predicted = torch.max(outputs.data, 1)

                    # Statistics
                    correct += torch.sum(predicted == labels.data)
                    total += labels.size(0)
                
                acc = torch.true_divide(correct, total)*100
                acc_history.append(acc)

        # Monitor  and plot indicators over epochs

        if verbose is True:
            with out:
                clear_output(wait=True)
                print(f'Epoch: {epoch+1}\n'
                      f'train loss / n_iter: {train_loss/train_n_iter}\n'
                      f'valid loss / n_iter: {valid_loss/valid_n_iter}')
                if monitor_acc is True:
                    print(f'accuracy on the test set: {acc:.3f}%.')
                x = range(1, epoch+2)
                if 'line_main' in locals():
                    line_main.remove()
                    line_par1.remove()
                line_main, = ax_main.plot(x, train_loss_history,
                                          label='Train loss', c='aqua')
                line_par1, = par1.plot(x, valid_loss_history, label='Valid loss',
                                      c='yellow')
                if monitor_acc is True:
                    if 'line_par2' in locals(): line_par2.remove()
                    line_par2, = par2.plot(x, acc_history, label='Accuracy',
                                           c='lime')
                    
                fig.legend(bbox_to_anchor=(2, 0.9), borderaxespad=0,
                   framealpha=0, bbox_transform=ax_main.transAxes)

                # Plot the evolution of the main learning rate over an epoch
                if monitor_lr is True:
                    x_lr = range(len(train_loader))
                    if 'line_lr' in locals(): line_lr.remove()
                    line_lr, = ax_lr.plot(x_lr, lr_history, c='orange')

                display(fig)

    if verbose is True:
        print(f'\nEnd training. Duration: {time.time() - start_time:.2f} s.\n')
        plt.close()

    if monitor_acc is False:
        return model, train_loss_history, valid_loss_history
    else:
        return model, train_loss_history, valid_loss_history, acc_history

# LeNet
---

We use a basic 'LeNet 5' architecture to classify the images from the MNIST dataset.

![Alt Text](https://github.com/mila-iqia/ecole_dl_mila_ivado/blob/master/tutoriaux/CNN/images/lenet5.png?raw=true)

## Creation and test of the model

We define the architecture of LeNet's neural network in the LeNet class. The constructor defines the layers and blocks of layers of the CNN, while the 'forward' method passes the input through them and returns the resulting output.

In [ ]:
class LeNet(nn.Module):

    def __init__(self):

        super(LeNet, self).__init__()

        self.block1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.block2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.fc = nn.Linear(7*7*32, 10)

    def forward(self, x):

        # We keep the softmax out of the neural network class
        out = self.block1(x)
        out = self.block2(out)
        # Flatten the output of block2
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

model = LeNet()
model.to(device)

print(model)
print("Parameters: ", sum([param.nelement() for param in model.parameters()]))

LeNet(
  (block1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=1568, out_features=10, bias=True)
)
Parameters:  28938


We first create the data loaders

In [ ]:
train_loader, valid_loader, test_loader = generate_data(batch_size=100)

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Over ten epochs, we train the model on the training set and evaluate its performance on the validation set to check for overfitting.

We define the criterion used to compute the loss (here the cross-entropy) as well as the optimizer used to update the weights (here Adam with a learning rate of 1e-2).

In [ ]:
num_epochs = 10
model, tlh, vlh, ah = train_test_MNIST(LeNet, num_epochs, learning_rate=1e-2,
                                  monitor_acc=True, verbose=True)


Start training.



Output()


End training. Duration: 55.77 s.



We plot the training and validation losses averaged over their respective number of iterations (the size of a batch), as well as the accuracy on the test set. The model seems to overfit since the average train loss keeps decreasing while the validation loss does not. This is likely due to the lack of a regularization mechanism to prevent the model to overfit the training data.

The accuracy is about 98%.

## Addition of Batch Normalization

We add batch normalization layers after convolution layers to help regularization in our model. The number of additional parameters is negligible and so is the additional training time over 10 epochs.

In [ ]:
class LeNetBN(nn.Module):

    def __init__(self):

        super(LeNetBN, self).__init__()

        # We add a batch normalization operation after the convolution
        self.block1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        # We add a batch normalization operation after the convolution
        self.block2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.fc = nn.Linear(7*7*32, 10)

    def forward(self, x):

        # We keep the softmax out of the neural network class
        out = self.block1(x)
        out = self.block2(out)
        # Flatten the output of block2
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

model = LeNetBN()
model.to(device)

print(model)
print("Parameters: ", sum([param.nelement() for param in model.parameters()]))

LeNetBN(
  (block1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=1568, out_features=10, bias=True)
)
Parameters:  29034


In [ ]:
num_epochs = 10
model, tlh, vlh, ah = train_test_MNIST(LeNetBN, num_epochs, learning_rate=1e-2,
                                  monitor_acc=True, verbose=True)


Start training.



Output()


End training. Duration: 58.07 s.



Now the validation loss decreases more regularly with time, and the accuracy progresses more regularly as well. We easily reach accuracies of 98.5-99% over ten epochs.

## Simple Dropout replacing or combined with Batch Normalization

We compare the use of Dropout layers after convolutions and the fully-connected layer as a mean of regularization instead of batch normalization.

In [ ]:
class LeNetDrop(nn.Module):

    def __init__(self):

        super(LeNetDrop, self).__init__()

        self.block1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.Dropout2d(0.125),
            nn.MaxPool2d(2)
        )

        self.block2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.Dropout2d(0.125),
            nn.MaxPool2d(2)
        )

        self.fc_drop = nn.Sequential(
            nn.Linear(7*7*32, 10),
            nn.Dropout(0.4)
        )

    def forward(self, x):

        # We keep the softmax out of the neural network class
        out = self.block1(x)
        out = self.block2(out)
        # Flatten the output of block2
        out = out.view(out.size(0), -1)
        out = self.fc_drop(out)
        return out

model = LeNetDrop()
model.to(device)

print(model)
print("Parameters: ", sum([param.nelement() for param in model.parameters()]))

LeNetDrop(
  (block1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): Dropout2d(p=0.125, inplace=False)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): Dropout2d(p=0.125, inplace=False)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_drop): Sequential(
    (0): Linear(in_features=1568, out_features=10, bias=True)
    (1): Dropout(p=0.4, inplace=False)
  )
)
Parameters:  28938


In [ ]:
num_epochs = 10
model, tlh, vlh, ah = train_test_MNIST(LeNetDrop, num_epochs, learning_rate=1e-2,
                                  monitor_acc=True, verbose=True)


Start training.



Output()


End training. Duration: 57.06 s.



Dropout requires more training time in order for the network to learn to perform with only a random subset of its connections. In order to fairly compare the two methods, we repeat the test over 50 epochs.

Batch Normalization:

In [ ]:
num_epochs = 50
model, tlh, vlh, ah = train_test_MNIST(LeNetBN, num_epochs, learning_rate=1e-2,
                                  monitor_acc=True, verbose=True)


Start training.



Output()


End training. Duration: 299.66 s.



Dropout:

In [ ]:
num_epochs = 50
model, tlh, vlh, ah = train_test_MNIST(LeNetDrop, num_epochs, learning_rate=1e-2,
                                  monitor_acc=True, verbose=True)


Start training.



Output()


End training. Duration: 292.78 s.



Batch normalization seems to perform better and is a bit more stable, although its validation loss increases with time.

The training loss is significantly larger than the validation loss on the Dropout model. This is due to the fact that Dropout is only applied during training in order to improve our model capacity: we sacrifice training accuracy in order to improve valid and test accuracy.

### Combining Dropout and Batch Normalization

In [ ]:
class LeNetDropBN(nn.Module):

    def __init__(self):

        super(LeNetDropBN, self).__init__()

        self.block1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Dropout2d(0.125),
            nn.MaxPool2d(2)
        )

        self.block2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout2d(0.125),
            nn.MaxPool2d(2)
        )

        self.fc_drop = nn.Sequential(
            nn.Linear(7*7*32, 10),
            nn.Dropout(0.4)
        )

    def forward(self, x):

        # We keep the softmax out of the neural network class
        out = self.block1(x)
        out = self.block2(out)
        # Flatten the output of block2
        out = out.view(out.size(0), -1)
        out = self.fc_drop(out)
        return out

model = LeNetDropBN()
model.to(device)

#print(model)
print("Parameters: ", sum([param.nelement() for param in model.parameters()]))

Parameters:  29034


In [ ]:
num_epochs = 50
model, tlh, vlh, ah = train_test_MNIST(LeNetDropBN, num_epochs, learning_rate=1e-2,
                                  monitor_acc=True, verbose=True)


Start training.



Output()


End training. Duration: 308.85 s.



This method provides the highest and most stable accuracy so far.

## Deepening the LeNet architecture
---

### Deeper layers

We use deeper convolution layers (of depth 32 and 64 respectively). This substantially increases the number of parameters of the model.

In [ ]:
class LeNetDL(nn.Module):

    def __init__(self):

        super(LeNetDL, self).__init__()

        self.block1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout2d(0.125),
            nn.MaxPool2d(2)
        )

        self.block2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout2d(0.125),
            nn.MaxPool2d(2)
        )

        self.fc_drop = nn.Sequential(
            nn.Linear(7*7*64, 10),
            nn.Dropout(0.4)
        )

    def forward(self, x):

        # We keep the softmax out of the neural network class
        out = self.block1(x)
        out = self.block2(out)
        # Flatten the output of block2
        out = out.view(out.size(0), -1)
        out = self.fc_drop(out)
        return out

model = LeNetDL()
model.to(device)

#print(model)
print("Parameters: ", sum([param.nelement() for param in model.parameters()]))

Parameters:  83658


In [ ]:
num_epochs = 50
model, tlh, vlh, ah = train_test_MNIST(LeNetDL, num_epochs, learning_rate=1e-2,
                                  monitor_acc=True, verbose=True)

The increase in training time over 50 epochs is about +30%. The performance is comparable yet slightly less stable, the network seems to overfit. 

We use a smaller kernel size (3) on the deeper layer.

In [ ]:
class LeNetDL(nn.Module):

    def __init__(self):

        super(LeNetDL, self).__init__()

        self.block1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout2d(0.125),
            nn.MaxPool2d(2)
        )

        self.block2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout2d(0.125),
            nn.MaxPool2d(2)
        )

        self.fc_drop = nn.Sequential(
            nn.Linear(7*7*64, 10),
            nn.Dropout(0.4)
        )

    def forward(self, x):

        # We keep the softmax out of the neural network class
        out = self.block1(x)
        out = self.block2(out)
        # Flatten the output of block2
        out = out.view(out.size(0), -1)
        out = self.fc_drop(out)
        return out

model = LeNetDL()
model.to(device)

#print(model)
print("Parameters: ", sum([param.nelement() for param in model.parameters()]))

Parameters:  50890


In [ ]:
num_epochs = 50
model, tlh, vlh, ah = train_test_MNIST(LeNetDL, num_epochs, learning_rate=1e-2,
                                  monitor_acc=True, verbose=True)


Start training.



Output()


End training. Duration: 321.24 s.



A first shallow layer with large kernel size followed by a second deeper layer with smaller kernel size seems to provide better results than the previous experiment, comparable with those of the shallower model.

Such a model with more parameters might benefit from longer training, but it still seems to start overfitting at some point.

### Adding layers to the architecture

In [ ]:
class LeNetDeep(nn.Module):

    def __init__(self):

        super(LeNetDeep, self).__init__()

        self.block1 = nn.Sequential(
            nn.BatchNorm2d(1),
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Dropout2d(0.125),
            nn.MaxPool2d(2)
        )

        self.block2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout2d(0.125),
            nn.MaxPool2d(2)
        )

        self.block3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout2d(0.125),
            nn.MaxPool2d(2)
        )

        self.block4 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout2d(0.125),
            nn.MaxPool2d(2)
        )

        self.fc_drop = nn.Sequential(
            nn.Linear(1*1*128, 10),
            nn.Dropout(0.4)
        )

    def forward(self, x):

        # We keep the softmax out of the neural network class
        out = self.block1(x)
        out = self.block2(out)
        out = self.block3(out)
        out = self.block4(out)
        # Flatten the output of block2
        out = out.view(out.size(0), -1)
        out = self.fc_drop(out)
        return out

model = LeNetDeep()
model.to(device)

#print(model)
print("Parameters: ", sum([param.nelement() for param in model.parameters()]))

Parameters:  107372


In [ ]:
num_epochs = 50
model, tlh, vlh, ah = train_test_MNIST(LeNetDeep, num_epochs, learning_rate=1e-2,
                                  monitor_acc=True, verbose=True)


Start training.



Output()


End training. Duration: 357.91 s.



The validation loss looks a bit better, more depth and training could help.

# Data Augmentation
---

The `visualize_sample` function allows us to have a look at pictures of the MNIST set in order to test the effect of transforms data normalization.

The three plots represent the MNIST pictures before normalization with grayscale [0, 1], after normalization with grayscale [-1, 1] and after normalization with grayscale [0, 1] respectively. We observe that the numbers on the right plot are thinner with sharper edges, which is a consequence of the truncation of negative values. This shows that the tensors contain values between -1 and 1. In addition, print statements allow to verify the minimal and maximal values in the tensors.

We can make further use of `visualize_sample` in order to see the effect of data augmentation methods on MNIST pictures, such as random cropping, rotation, zone erasingn, etc.

In [ ]:
def visualize_sample(n_sample, transform):

    ref_dataset = MNIST(root='../data', 
                     train=False, 
                     transform=transforms.ToTensor())
    trans_dataset = MNIST(root='../data', 
                     train=False, 
                     transform=transform)

    batch_size = 1

    ref_loader = DataLoader(dataset=ref_dataset,
                            sampler=ChunkSampler(n_sample, 0),
                            batch_size=batch_size,
                            shuffle=False)
    trans_loader = DataLoader(dataset=trans_dataset,
                              sampler=ChunkSampler(n_sample, 0),
                              batch_size=batch_size,
                              shuffle=False)

    iter_ref, iter_trans = cycle(ref_loader), cycle(trans_loader)
    count1, count2 = cycle(range(1, n_sample+1)), cycle(range(1, n_sample+1))
    out = widgets.Output()
    display(out)

    plt.style.use('dark_background')

    for ref, trans in zip(iter_ref, iter_trans):
        try:
            imref, imtrans = ref[0].numpy()[0, 0, :, :], trans[0].numpy()[0, 0, :, :]
            with out:
                clear_output(wait=True)
                fig = plt.figure(figsize=(12, 8))
                ax1 = fig.add_subplot(131)
                ax1.set_title('ref, scale: [0, 1]')
                ax2 = fig.add_subplot(132)
                ax2.set_title('trans, scale: [-1, 1]')
                ax3 = fig.add_subplot(133)
                ax3.set_title('trans, scale: [0, 1]')
                print(f'ref: count: {next(count1)}, min: {ref[0].min()}, max: {ref[0].max()}')
                print(f'trans: count: {next(count2)}, min: {trans[0].min()}, max: {trans[0].max()}\n')
                ax1.imshow(imref, cmap='gray', vmin=0, vmax=1)
                ax2.imshow(imtrans, cmap='gray', vmin=-1, vmax=1)
                ax3.imshow(imtrans, cmap='gray', vmin=0, vmax=1)
                plt.show()
            time.sleep(2)
        except:
            print('\nLoop interruption.')
            break

We define a transform that standardizes the MNIST pictures (values range between -1 and 1) which can accelerate the convergence of neural networks. The transform is applied at the definition of the MNIST train and test datasets.

Notice that Pytorch's `ToTensor` method already normalizes the content of inputs between 0 and 1, which can be sufficient.

In [ ]:
mnist_norm = transforms.Compose([
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5,), (0.5,))
                                 ])

In [ ]:
visualize_sample(20, mnist_norm)

Output()


Loop interruption.


We use different sensible transformations of our MNIST pictures in order to augment the data.

Let us note that PyTorch performs online data augmentation, i.e. augmentation 'on the fly'. At each epoch, the data are randomly transformed according to the list of passed transformation. Therefore in order to actually increase the data and not just modify them, we need to increase the number of training epochs. The original dataset does not need any modification (as is done in static or offline augmentation).

In [ ]:
mnist_aug = transforms.Compose([
                                transforms.RandomResizedCrop(28, scale=(0.8, 1.0)),
                                transforms.RandomRotation(30),
                                transforms.ToTensor(),
                                transforms.RandomErasing(p=0.5, scale=(0.02, 0.2)),
                                transforms.Normalize((0.5,), (0.5,))
                                ])

In [ ]:
visualize_sample(20, mnist_aug)

Output()


Loop interruption.


In [ ]:
train_loader, valid_loader, test_loader = generate_data(batch_size=100,
                                                        transform=mnist_aug)
num_epochs = 20
model, tlh, vlh, ah = train_test_MNIST(LeNetDeep, num_epochs, learning_rate=1e-2,
                                  monitor_acc=True, verbose=True)


Start training.



Output()


End training. Duration: 349.87 s.



Although both the training and validation loss appear to be steadily decreasing (albeit with higher values than models without augmentation), the test accuracy is strongly unstable.

We keep the data-augmenting transforms but remove the additional normalization of the tensors and keep the values between 0 and 1:

In [ ]:
mnist_aug = transforms.Compose([
                                transforms.RandomResizedCrop(28, scale=(0.8, 1.0)),
                                transforms.RandomRotation(30),
                                transforms.ToTensor(),
                                transforms.RandomErasing(p=0.5, scale=(0.02, 0.2)),
                                #transforms.Normalize((0.5,), (0.5,))
                                ])

In [ ]:
train_loader, valid_loader, test_loader = generate_data(batch_size=100,
                                                        transform=mnist_aug)
num_epochs = 100
model, tlh, vlh, ah = train_test_MNIST(LeNetDeep, num_epochs, learning_rate=1e-2,
                                  monitor_acc=True, verbose=True)


Start training.



Output()


End training. Duration: 1792.98 s.



Although both losses steadily decrease, the progress in accuracy is slow and appears to encounter a resistance at still low values.

We also remove the RandomErasing transform that is a source of unstable and limited progress in this scenario, and increase the resizing range to compensate:

In [ ]:
mnist_aug = transforms.Compose([
                                transforms.RandomResizedCrop(28, scale=(0.8, 1.2)),
                                transforms.RandomRotation(30),
                                transforms.ToTensor(),
                                #transforms.RandomErasing(p=0.5, scale=(0.02, 0.2)),
                                #transforms.Normalize((0.5,), (0.5,))
                                ])

In [ ]:
train_loader, valid_loader, test_loader = generate_data(batch_size=100,
                                                        transform=mnist_aug)
num_epochs = 100
model, tlh, vlh, ah = train_test_MNIST(LeNetDeep, num_epochs, learning_rate=1e-2,
                                  monitor_acc=True, verbose=True)


Start training.



Output()


End training. Duration: 1233.00 s.



We reach accuracies comparable to that of LeNetDeep without data augmentation, despite using more epochs. We need to find another way to improve the learning process.

# Cyclical learning rate

---

## Cosine annealing rate

We try using a cosine annealing learning rate to improve the convergence of our model. Cyclical learning rates allow the optimizer to avoid staying stuck in steep minima by periodically taking large steps and then reducing the step size again to find the best point of a new, hopefully wider minimum. Indeed, if the optimizer finds a wide minimum, it will likely stay inside it even when taking large steps.

Wide minima are preferred as they generalize better than narrow ones, since their own topology is relatively less affected by the intrinsic error of the represented loss surface due to lack of data.

The cycle period should be larger than the batch size. If signficantly smaller, the oscillations of the learning rate will bring no benefit as the effective rate will just be equivalent to their average. If the cycle period is very large, one needs a very large number of epochs in order to cover a significant number of cycles.

We will use cycle periods up to 10 times larger than the batch size as a sensible compromise between speed and accuracy.

In [ ]:
train_loader, valid_loader, test_loader = generate_data(batch_size=100)
num_epochs = 50
model, tlh, vlh, ah = train_test_MNIST(LeNetDeep, num_epochs, learning_rate=1e-2,
                                  monitor_acc=True,
                                  scheduler=CosineAnnealingLR, monitor_lr=True,
                                  verbose=True, T_max=1000)


Start training.



Output()


End training. Duration: 364.01 s.



The maximal learning rate seems too large to visualize a steady long-term increase in accuracy. We reduce it by an order of magnitude.

In [ ]:
train_loader, valid_loader, test_loader = generate_data(batch_size=100)
num_epochs = 50
model, tlh, vlh, ah = train_test_MNIST(LeNetDeep, num_epochs, learning_rate=1e-3,
                                  monitor_acc=True,
                                  scheduler=CosineAnnealingLR, monitor_lr=True,
                                  verbose=True, T_max=1000)


Start training.



Output()


End training. Duration: 364.05 s.



In [ ]:
print(f'The maximal accuracy reached is: {max(ah).item():.2f}%.')

The maximal accuracy reached is: 99.48%.


We get encouraging results on the accuracy that reaches high values. The valid loss starts increasing around 30 epochs.

We now combine the cyclical learning rate with data augmentation:

In [ ]:
train_loader, valid_loader, test_loader = generate_data(batch_size=100,
                                                        transform=mnist_aug)
num_epochs = 50
model, tlh, vlh, ah = train_test_MNIST(LeNetDeep, num_epochs, learning_rate=1e-3,
                                  monitor_acc=True,
                                  scheduler=CosineAnnealingLR, monitor_lr=True,
                                  verbose=True, T_max=1000)


Start training.



Output()


End training. Duration: 604.51 s.



In [ ]:
print(f'The maximal accuracy reached is: {max(ah).item():.2f}%.')

The maximal accuracy reached is: 99.45%.


Although the maximal accuracy is similar, we observe that with data augmentation, the valid loss does not start increasing past 30 epochs. This means we can use longer training with the hope that our model's accuracy will keep increasing.

In [ ]:
train_loader, valid_loader, test_loader = generate_data(batch_size=250,
                                                        transform=mnist_aug)
num_epochs = 80
model, tlh, vlh, ah = train_test_MNIST(LeNetDeep, num_epochs, learning_rate=1e-3,
                                  monitor_acc=True,
                                  scheduler=CosineAnnealingLR, monitor_lr=True,
                                  verbose=True, T_max=1000)


Start training.



Output()


End training. Duration: 890.46 s.



In [ ]:
print(f'The maximal accuracy reached is: {max(ah).item():.2f}%.')

The maximal accuracy reached is: 99.55%.


We obtain the highest accuracy observed so far.

## Triangular learning rate

We try the 'triangular' mode of the `CyclicLR` scheduler:

In [ ]:
num_epochs, batch_size = 100, 250

train_loader, valid_loader, test_loader = generate_data(batch_size=250,
                                                        transform=mnist_aug)

model, tlh, vlh, ah = train_test_MNIST(LeNetDeep, num_epochs, learning_rate=1e-3,
                                  monitor_acc=True,
                                  scheduler=CyclicLR, monitor_lr=True,
                                  verbose=True, base_lr=1e-4, max_lr=1e-3,
                                  step_size_up=10*batch_size, mode='triangular',
                                  cycle_momentum=False)


Start training.



Output()


End training. Duration: 1112.25 s.



In [ ]:
print(f'The maximal accuracy reached is: {max(ah).item():.2f}%.')

The maximal accuracy reached is: 99.55%.


In [ ]:
num_epochs, batch_size = 100, 250

train_loader, valid_loader, test_loader = generate_data(batch_size=250,
                                                        transform=mnist_aug)

model, tlh, vlh, ah = train_test_MNIST(LeNetDeep, num_epochs, learning_rate=1e-3,
                                  monitor_acc=True,
                                  scheduler=CyclicLR, monitor_lr=True,
                                  verbose=True, base_lr=1e-4, max_lr=1e-3,
                                  step_size_up=4*batch_size, mode='triangular',
                                  cycle_momentum=False)


Start training.



Output()


End training. Duration: 1109.60 s.



In [ ]:
print(f'The maximal accuracy reached is: {max(ah).item():.2f}%.')

The maximal accuracy reached is: 99.58%.


We obtain similar results with the triangular scheduler.

In [ ]:
num_epochs, batch_size = 100, 250

train_loader, valid_loader, test_loader = generate_data(batch_size=250,
                                                        transform=mnist_aug)

model, tlh, vlh, ah = train_test_MNIST(LeNetDeep, num_epochs, learning_rate=1e-2,
                                  monitor_acc=True,
                                  scheduler=CyclicLR, monitor_lr=True,
                                  verbose=True, base_lr=1e-4, max_lr=1e-2,
                                  step_size_up=10*batch_size, mode='triangular2',
                                  cycle_momentum=False)


Start training.



Output()


End training. Duration: 1114.69 s.



In [ ]:
print(f'The maximal accuracy reached is: {max(ah).item():.2f}%.')

The maximal accuracy reached is: 99.55%.


The 'triangular2' mode of the `CyclicLR` scheduler is a triangular profile which amplitude is divided by 2 for each completed cycle. We started it with a larger learning rate to accelerate convergence before the cycle divison brings it back to smaller amplitudes. The results are similar to other procedures.

## Cosine annealing with warm restarts

We try the `CosineAnnealingWarmRestarts` scheduler where the learning rate "jumps" to its maximal value before going down following a cosine trend. The `T_mult` optional argument allows to multiply the cycle period by an integer number after every completed cycle.

In [ ]:
train_loader, valid_loader, test_loader = generate_data(batch_size=250,
                                                        transform=mnist_aug)
num_epochs = 100
model, tlh, vlh, ah = train_test_MNIST(LeNetDeep, num_epochs, learning_rate=1e-3,
                                  monitor_acc=True,
                                  scheduler=CosineAnnealingWarmRestarts, monitor_lr=True,
                                  verbose=True, T_0=250, T_mult=2)


Start training.



Output()


End training. Duration: 1114.43 s.



In [ ]:
print(f'The maximal accuracy reached is: {max(ah).item():.2f}%.')

The maximal accuracy reached is: 99.51%.


We are stuck in terms of performance on the accuracy metric. We already reach near state-of-the-art accuracy. But in order to go further, we probably need an architecture deeper than the one we have been using until then.

# Combining a deeper architecture with optimization methods for optimal results

We use a deeper architecture called `SimpleNet` inspired from the published model with the same name, although it is a more basic version. We try to optimize training in order to get optimal accuracy on the test set with the available resources in a relatively short time.

In [ ]:
 class SimpleNet(nn.Module):

    def __init__(self):
        super(SimpleNet, self).__init__()

        self.block1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU()
        )
        self.block2 = nn.Sequential( 
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),
            nn.ReLU()
        )

        self.block3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU()
        )

        self.block4 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),
            nn.ReLU()
        )

        self.fc = nn.Sequential(
            nn.Linear(7*7*64, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)       
        x = x.view(x.size(0), -1)
        x = self.fc(x)       
        return x

model = SimpleNet()
model.to(device)

print("Parameters: ", sum([param.nelement() for param in model.parameters()]))

Parameters:  1677290


In [ ]:
train_loader, valid_loader, test_loader = generate_data(batch_size=250,
                                                        transform=mnist_aug)
num_epochs = 100
model, tlh, vlh, ah = train_test_MNIST(SimpleNet, num_epochs, learning_rate=1e-3,
                                  monitor_acc=True,
                                  scheduler=CosineAnnealingWarmRestarts, monitor_lr=True,
                                  verbose=True, T_0=1000, T_mult=1)


Start training.



Output()


End training. Duration: 1395.70 s.



In [ ]:
print(f'The maximal accuracy reached is: {max(ah).item():.2f}%.')

The maximal accuracy reached is: 99.69%.


Error rates typically reach down 0.4%, with a record value of 0.31%. The average valid loss appears to keep decreasing, meaning our model does not overfit. The training takes about 25 min on a Colab GPU.